In [23]:
import pandas as pd
import numpy as np
import torch 
import torch.nn as  nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
import time
from tqdm.autonotebook import tqdm, trange
import matplotlib.pyplot as plt
import seaborn as sns

device = 'cuda' if torch.cuda.is_available() else 'cpu'

companies = ['GAZP', 'SBER', 'LKOH', 'GMKN', 'MGNT',
       'TATN', 'NVTK', 'SNGS', 'PLZL', 'SNGSP',
       'PIKK', 'ROSN', 'SBERP', 'CHMF', 'NLMK',
       'IRAO', 'YNDX', 'ALRS', 'RUAL', 'MTSS',
       'MAGN', 'PHOR', 'RTKM', 'VTBR', 'TATNP',
       'TCSG', 'AGRO', 'OZON', 'AFLT', 'AFKS',
       'FEES', 'TRNFP', 'CBOM', 'VKCO', 'ENPG',
       'SGZH', 'FIVE', 'POLY', 'MOEX', 'GLTR',
       'HYDR', 'FIXP']


def hour2time(hour):
    if hour == 1:
        res = '11:00:00'
    elif hour == 2:
        res = '12:00:00'
    elif hour == 3:
        res = '13:00:00'
    elif hour == 4:
        res = '14:00:00'
    elif hour == 5:
        res = '15:00:00'
    elif hour == 6:
        res = '16:00:00'
    elif hour == 7:
        res = '17:00:00'
    elif hour == 8:
        res = '18:00:00'
    elif hour == 9:
        res = '18:45:00'
    return res

In [2]:
path = '~/Рабочий стол/ДИПЛОМ НОУТБУКИ/DL forcasitng diplom/Данные Мосбиржа/Total_MOEX_(NEW)hourly_with_time_df'
total_df = pd.read_csv(path, index_col='tradedate')

In [4]:
total_df = total_df[ ('2023-02-03'<=total_df.index) * (total_df.index<= '2023-10-30')]

In [146]:
total_df['date'] = total_df.index

res = []
for i in range(total_df.shape[0]):
    res.append(np.mean((total_df.close[i], total_df.open[i]) ) )
total_df['mean_price'] = res   
total_df['vol_in_rub'] = total_df.vol * total_df.mean_price

In [148]:
total_df

res = [1]
h = 2
for i in range(1, 68639):
    if total_df.iloc[i,3] == total_df.iloc[i-1,3]:
        res.append(h)
        h+=1
    if total_df.iloc[i,3] != total_df.iloc[i-1,3]:
        h=1
        res.append(h)
        h+=1
total_df['hour'] = res

In [149]:
volumes = total_df.groupby('date').aggregate({'vol_in_rub':'sum'})

In [150]:
volumes.to_csv('market_volumes')

# Closes

In [61]:
df = pd.read_csv('backtest_data').rename(columns={'Unnamed: 0':'date'})
df.head(1)

,date,Asset,Pred,Fact,date,hour,pct_pred,pct_fact
0,2023-02-06,GAZP,157.116934,156.369999,2023-02-06,1,0.0,0.0


In [62]:
dd = df['date'].iloc[:,1]
df = df.drop(columns='date')
df['date'] = dd

In [63]:
bad_days = df.groupby('date').aggregate({'Asset':'count'}).query("Asset < 200").index

In [64]:
for i in range(0,len(bad_days)):    
    df = df[df.date != bad_days[i]]

In [65]:
df.date

0        2023-02-06
1        2023-02-06
2        2023-02-07
3        2023-02-07
4        2023-02-07
            ...    
66263    2023-10-24
66264    2023-10-24
66265    2023-10-24
66266    2023-10-24
66267    2023-10-24
Name: date, Length: 65181, dtype: object

In [66]:
df['Date'] = pd.to_datetime(df.date + ' ' + df.hour.apply(hour2time) )
df.drop(['date','hour','pct_pred','pct_fact','Pred'],1,inplace=True)

/tmp/ipykernel_8980/3486096165.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['date','hour','pct_pred','pct_fact','Pred'],1,inplace=True)


In [67]:
pivot_df = df.pivot(index='Date', columns='Asset', values='Fact')


# Заменим названия колонок для удобства
pivot_df.columns.name = None

In [127]:
pivot_df.to_csv('cvxportfolio_fact_closes')

# Open

In [85]:
df = pd.read_csv('additional_info')
df.head(1)

,tradedate,secid,open,close,date,hour
0,2020-01-03,GAZP,259.43,259.1,2020-01-03,1


In [88]:
df = df[df['date']>= '2023-02-03']
df = df[df['date']<= '2023-10-24']

In [90]:
df['Date'] = pd.to_datetime(df.date + ' ' + df.hour.apply(hour2time) )

In [91]:
df.head(1)

,tradedate,secid,open,close,date,hour,Date
6768,2023-02-03,GAZP,158.54,158.68,2023-02-03,1,2023-02-03 11:00:00


In [92]:
df.drop(['date','hour','close','tradedate'],1,inplace=True)

/tmp/ipykernel_8980/2395446014.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['date','hour','close','tradedate'],1,inplace=True)


In [93]:
pivot_df = df.pivot(index='Date', columns='secid', values='open')


# Заменим названия колонок для удобства
pivot_df.columns.name = None

In [95]:
pivot_df.to_csv('cvxportfolio_fact_opens')

# Returns

In [12]:
df = pd.read_csv('backtest_data').rename(columns={'Unnamed: 0':'date'})

In [13]:
dd = df['date'].iloc[:,1]
df = df.drop(columns='date')
df['date'] = dd

In [14]:
df = df[df['date']>= '2023-02-03']
df = df[df['date']<= '2023-10-24']

In [16]:
bad_days = df.groupby('date').aggregate({'Asset':'count'}).query("Asset < 200").index

In [17]:
for i in range(0,len(bad_days)):    
    df = df[df.date != bad_days[i]]

In [19]:
df['Date'] = pd.to_datetime(df.date + ' ' + df.hour.apply(hour2time) )

df.drop(['date','hour','pct_pred','Pred','Fact'],1,inplace=True)

/tmp/ipykernel_6681/2043738883.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df.drop(['date','hour','pct_pred','Pred','Fact'],1,inplace=True)


In [21]:
pivot_df = df.pivot(index='Date', columns='Asset', values='pct_fact')


# Заменим названия колонок для удобства
pivot_df.columns.name = None

In [27]:
pivot_df.to_csv('cvxportfolio_fact_returns')